# Empirical verification of the theoretical value of expectations and variances

This notebook contain the experiment described in Section 5.1 of the paper. It seeks to verify the theoretical value of expectations and variances under independent Gaussian distributed assumptions is within an accepted range specified by a bootstrap sampling confidence interval.

In [1]:
import numpy as np
from normal_normal_model import E_V, var_V, get_prioritisation_value_samples
from util import get_bootstrap_mean, get_bootstrap_var
from random import sample 

In [2]:
def print_run_setting(count, N, M, mu_X, sigma_X, mu_eps, sigma_1, sigma_2):
    print("Run {}: N = {}, M = {}, mu_X = {}, sigma_X = {}, "
          "mu_epsilon = {}, sigma_1 = {}, sigma_2 = {}"
          .format(count, N, M, mu_X, sigma_X, mu_eps, sigma_1, sigma_2))
    
def EV_within_CI(bootstrap_samples, mu_X, sigma_sq_X, sigma_sq_eps, 
                 N, M, verbose=False, header=''):
    EV = E_V(mu_X, sigma_sq_X, sigma_sq_eps, N, M)
    return(within_CI(EV, bootstrap_samples,
                     verbose=verbose, print_header=header + " - E(V)"))

def varV_within_CI(bootstrap_samples, sigma_sq_X, sigma_sq_eps, 
                   N, M, verbose=False, header=''):
    varV = var_V(sigma_sq_X, sigma_sq_eps, N, M)
    return(within_CI(varV, bootstrap_samples,
                     verbose=verbose, print_header=header + " - Var(V)"))
    
def exp_improvement_within_CI(bootstrap_samples, mu_X, sigma_sq_X, 
                              sigma_sq_1, sigma_sq_2, N, M, 
                              verbose=False, header=''):
    exp_improvement = (
        E_V(mu_X, sigma_sq_X, sigma_sq_2, N, M) -
        E_V(mu_X, sigma_sq_X, sigma_sq_1, N, M))
    return(within_CI(exp_improvement, bootstrap_samples,
                     verbose=verbose, print_header=header + "E(D)"))


def within_CI(theoretical_quantity, bootstrap_samples, 
              verbose=False, print_header=""):
    CI_low = np.percentile(bootstrap_samples, 2.5)
    CI_high = np.percentile(bootstrap_samples, 97.5)
    within_CI = ((theoretical_quantity >= CI_low) and 
                 (theoretical_quantity  <= CI_high))
    
    if verbose:
        print("{}: Theoretical quantity = {}, 95% CI = [{}, {}], "
              "Within CI: {}"
                  .format(print_header, 
                          np.round(theoretical_quantity, 4), 
                          np.round(CI_low, 4), np.round(CI_high, 4), 
                          str(within_CI)))
    
    return within_CI

In [3]:
# Constants
NUM_SAMPLES = 5000
NUM_BOOTSTRAPS = 1000

def run_verification(params):
    run_counter = 0
    noisy_EV_within_CI_counter = 0
    noisy_varV_within_CI_counter = 0
    clean_EV_within_CI_counter = 0
    clean_varV_within_CI_counter = 0
    improvement_within_CI_counter = 0

    for param in params:
        N = param['N']
        M = param['M']
        sigma_X = param['sigma_X']
        sigma_1 = param['sigma_1']
        sigma_2 = param['sigma_2']
        mu_X = param['mu_X']
        mu_epsilon = param['mu_epsilon']

        # Reconciling the use of sigmas by numpy
        # and sigma_sqs in the theoretical calculations
        sigma_sq_1 = sigma_1 ** 2
        sigma_sq_2 = sigma_2 ** 2
        sigma_sq_X = sigma_X ** 2

        run_counter+=1
        print_run_setting(run_counter, N, M, mu_X, sigma_X, 
                          mu_epsilon, sigma_1, sigma_2)

        noisy_mean, clean_mean, improvement = (
            get_prioritisation_value_samples(
                NUM_SAMPLES, N, M, mu_X, mu_epsilon, 
                sigma_sq_X, sigma_sq_1, sigma_sq_2, verbose=False)
        )

        noisy_bootstrap_mean = get_bootstrap_mean(noisy_mean, NUM_BOOTSTRAPS)
        if EV_within_CI(noisy_bootstrap_mean, mu_X, sigma_sq_X, 
                        sigma_sq_1, N, M, verbose=True, header="sigma_sq_1"):
            noisy_EV_within_CI_counter += 1

        noisy_bootstrap_var = get_bootstrap_var(noisy_mean, NUM_BOOTSTRAPS)
        if varV_within_CI(noisy_bootstrap_var, sigma_sq_X, 
                          sigma_sq_1, N, M, verbose=True, header="sigma_sq_1"):
            noisy_varV_within_CI_counter += 1

        clean_bootstrap_mean = get_bootstrap_mean(clean_mean, NUM_BOOTSTRAPS)
        if EV_within_CI(clean_bootstrap_mean, mu_X, sigma_sq_X, 
                        sigma_sq_2, N, M, verbose=True, header="sigma_sq_2"):
            clean_EV_within_CI_counter += 1

        clean_bootstrap_var = get_bootstrap_var(clean_mean, NUM_BOOTSTRAPS)
        if varV_within_CI(clean_bootstrap_var, sigma_sq_X, 
                          sigma_sq_2, N, M, verbose=True, header="sigma_sq_2"):
            clean_varV_within_CI_counter += 1

        improvement_bootstrap_mean = get_bootstrap_mean(improvement, NUM_BOOTSTRAPS)
        if exp_improvement_within_CI(improvement_bootstrap_mean, mu_X, sigma_sq_X, 
                                     sigma_sq_1, sigma_sq_2, N, M, verbose=True, header=""):
            improvement_within_CI_counter += 1

    #     improvement_bootstrap_var = get_bootstrap_var(improvement, NUM_BOOTSTRAPS)
    
        if (run_counter % 20 == 0):
            print("Number of experiments where theoretical quantity lies within bootstrap CI: \n"
                  "E(V) (sigma_sq_1): {}/{} \n"
                  "Var(V) (sigma_sq_1): {}/{} \n"
                  "E(V) (sigma_sq_2): {}/{} \n"
                  "Var(V) (sigma_sq_2): {}/{} \n"
                  "E(D): {}/{} \n"
                  .format(noisy_EV_within_CI_counter, run_counter,
                          noisy_varV_within_CI_counter, run_counter,
                          clean_EV_within_CI_counter, run_counter,
                          clean_varV_within_CI_counter, run_counter,
                          improvement_within_CI_counter, run_counter))

# Sampling experiment parameters from curated parameter space

In [4]:
param_space = [
    {'N': N, 'M': M, 'sigma_X': sigma_X, 'sigma_1': sigma_1,
     'sigma_2': sigma_2, 'mu_X': mu_X, 'mu_epsilon': mu_epsilon}
    
    for N in [50, 100, 500, 1000, 5000]
    for M in [int(N/10.0), int(2*N/10.0), int(3*N/10.0),
              int(4*N/10.0), int(5*N/10.0)]
    for sigma_X in [0.5, 1, 2, 3, 5]
    for sigma_1 in [0.2, 0.5, 1, 2, 3, 5]
    for sigma_2 in [sigma_1/5.0, 2*sigma_1/5.0, 
                    3*sigma_1/5.0, 4*sigma_1/5.0, 5*sigma_1/5.0]
    for mu_X in [-5, -1, 0, 1, 5]
    for mu_epsilon in [-5, -3, -1, 0, 1, 3, 5]
]

print("Cardinality of parameter space: {}".format(len(param_space)))

Cardinality of parameter space: 131250


In [5]:
run_verification(sample(param_space, 400))

Run 1: N = 50, M = 10, mu_X = -1, sigma_X = 1, mu_epsilon = 5, sigma_1 = 1, sigma_2 = 0.4
sigma_sq_1 - E(V): Theoretical quantity = -0.0295, 95% CI = [-0.0412, -0.0275], Within CI: True
sigma_sq_1 - Var(V): Theoretical quantity = 0.0717, 95% CI = [0.0689, 0.0748], Within CI: True
sigma_sq_2 - E(V): Theoretical quantity = 0.2743, 95% CI = [0.265, 0.2769], Within CI: True
sigma_sq_2 - Var(V): Theoretical quantity = 0.0512, 95% CI = [0.0489, 0.0529], Within CI: True
E(D): Theoretical quantity = 0.3038, 95% CI = [0.3008, 0.3107], Within CI: True
Run 2: N = 500, M = 250, mu_X = 1, sigma_X = 3, mu_epsilon = 3, sigma_1 = 1, sigma_2 = 0.6
sigma_sq_1 - E(V): Theoretical quantity = 3.2668, 95% CI = [3.2662, 3.2747], Within CI: True
sigma_sq_1 - Var(V): Theoretical quantity = 0.0256, 95% CI = [0.0241, 0.0262], Within CI: True
sigma_sq_2 - E(V): Theoretical quantity = 3.343, 95% CI = [3.3421, 3.3508], Within CI: True
sigma_sq_2 - Var(V): Theoretical quantity = 0.0249, 95% CI = [0.0234, 0.0254], Wi

sigma_sq_1 - E(V): Theoretical quantity = 4.9376, 95% CI = [4.9308, 4.9554], Within CI: True
sigma_sq_1 - Var(V): Theoretical quantity = 0.1829, 95% CI = [0.1816, 0.1966], Within CI: True
sigma_sq_2 - E(V): Theoretical quantity = 5.4526, 95% CI = [5.4471, 5.4703], Within CI: True
sigma_sq_2 - Var(V): Theoretical quantity = 0.1682, 95% CI = [0.166, 0.18], Within CI: True
E(D): Theoretical quantity = 0.5151, 95% CI = [0.5074, 0.5255], Within CI: True
Run 17: N = 500, M = 150, mu_X = 5, sigma_X = 2, mu_epsilon = 0, sigma_1 = 2, sigma_2 = 1.6
sigma_sq_1 - E(V): Theoretical quantity = 6.6358, 95% CI = [6.6363, 6.6439], Within CI: False
sigma_sq_1 - Var(V): Theoretical quantity = 0.0206, 95% CI = [0.0198, 0.0214], Within CI: True
sigma_sq_2 - E(V): Theoretical quantity = 6.8065, 95% CI = [6.8059, 6.8136], Within CI: True
sigma_sq_2 - Var(V): Theoretical quantity = 0.0192, 95% CI = [0.0184, 0.0199], Within CI: True
E(D): Theoretical quantity = 0.1706, 95% CI = [0.1667, 0.1723], Within CI: Tru

sigma_sq_2 - Var(V): Theoretical quantity = 0.0066, 95% CI = [0.0063, 0.0068], Within CI: True
E(D): Theoretical quantity = 0.3007, 95% CI = [0.2993, 0.3025], Within CI: True
Run 31: N = 100, M = 20, mu_X = 5, sigma_X = 0.5, mu_epsilon = 1, sigma_1 = 0.2, sigma_2 = 0.16
sigma_sq_1 - E(V): Theoretical quantity = 5.6431, 95% CI = [5.6403, 5.645], Within CI: True
sigma_sq_1 - Var(V): Theoretical quantity = 0.0066, 95% CI = [0.0065, 0.007], Within CI: True
sigma_sq_2 - E(V): Theoretical quantity = 5.6597, 95% CI = [5.6573, 5.6619], Within CI: True
sigma_sq_2 - Var(V): Theoretical quantity = 0.0063, 95% CI = [0.0062, 0.0067], Within CI: True
E(D): Theoretical quantity = 0.0166, 95% CI = [0.016, 0.0175], Within CI: True
Run 32: N = 50, M = 5, mu_X = -1, sigma_X = 1, mu_epsilon = 3, sigma_1 = 1, sigma_2 = 0.8
sigma_sq_1 - E(V): Theoretical quantity = 0.2069, 95% CI = [0.1978, 0.2179], Within CI: True
sigma_sq_1 - Var(V): Theoretical quantity = 0.1326, 95% CI = [0.1302, 0.141], Within CI: True

sigma_sq_1 - Var(V): Theoretical quantity = 0.0008, 95% CI = [0.0007, 0.0008], Within CI: True
sigma_sq_2 - E(V): Theoretical quantity = 5.3068, 95% CI = [5.3063, 5.3079], Within CI: True
sigma_sq_2 - Var(V): Theoretical quantity = 0.0007, 95% CI = [0.0007, 0.0008], Within CI: True
E(D): Theoretical quantity = 0.0479, 95% CI = [0.047, 0.0485], Within CI: True
Run 46: N = 5000, M = 2500, mu_X = -5, sigma_X = 1, mu_epsilon = -3, sigma_1 = 5, sigma_2 = 5.0
sigma_sq_1 - E(V): Theoretical quantity = -4.8436, 95% CI = [-4.8442, -4.8431], Within CI: True
sigma_sq_1 - Var(V): Theoretical quantity = 0.0004, 95% CI = [0.0004, 0.0004], Within CI: False
sigma_sq_2 - E(V): Theoretical quantity = -4.8436, 95% CI = [-4.844, -4.8429], Within CI: True
sigma_sq_2 - Var(V): Theoretical quantity = 0.0004, 95% CI = [0.0004, 0.0004], Within CI: True
E(D): Theoretical quantity = 0.0, 95% CI = [-0.0003, 0.0008], Within CI: True
Run 47: N = 1000, M = 100, mu_X = 5, sigma_X = 5, mu_epsilon = 5, sigma_1 = 0.5, s

sigma_sq_1 - Var(V): Theoretical quantity = 0.0006, 95% CI = [0.0006, 0.0006], Within CI: True
sigma_sq_2 - E(V): Theoretical quantity = 5.1856, 95% CI = [5.1845, 5.1859], Within CI: True
sigma_sq_2 - Var(V): Theoretical quantity = 0.0006, 95% CI = [0.0006, 0.0006], Within CI: True
E(D): Theoretical quantity = 0.0685, 95% CI = [0.0682, 0.0696], Within CI: True
Number of experiments where theoretical quantity lies within bootstrap CI: 
E(V) (sigma_sq_1): 57/60 
Var(V) (sigma_sq_1): 56/60 
E(V) (sigma_sq_2): 57/60 
Var(V) (sigma_sq_2): 53/60 
E(D): 57/60 

Run 61: N = 5000, M = 1500, mu_X = -1, sigma_X = 1, mu_epsilon = 1, sigma_1 = 5, sigma_2 = 2.0
sigma_sq_1 - E(V): Theoretical quantity = -0.7728, 95% CI = [-0.7735, -0.772], Within CI: True
sigma_sq_1 - Var(V): Theoretical quantity = 0.0007, 95% CI = [0.0006, 0.0007], Within CI: True
sigma_sq_2 - E(V): Theoretical quantity = -0.4818, 95% CI = [-0.4826, -0.4812], Within CI: True
sigma_sq_2 - Var(V): Theoretical quantity = 0.0006, 95% CI

...

sigma_sq_2 - Var(V): Theoretical quantity = 0.033, 95% CI = [0.0325, 0.0352], Within CI: True
E(D): Theoretical quantity = 0.0168, 95% CI = [0.0162, 0.017], Within CI: True
Run 266: N = 500, M = 50, mu_X = 0, sigma_X = 1, mu_epsilon = -1, sigma_1 = 3, sigma_2 = 3.0
sigma_sq_1 - E(V): Theoretical quantity = 0.5531, 95% CI = [0.5469, 0.5547], Within CI: True
sigma_sq_1 - Var(V): Theoretical quantity = 0.0187, 95% CI = [0.0176, 0.019], Within CI: True
sigma_sq_2 - E(V): Theoretical quantity = 0.5531, 95% CI = [0.5474, 0.5547], Within CI: True
sigma_sq_2 - Var(V): Theoretical quantity = 0.0187, 95% CI = [0.0184, 0.02], Within CI: True
E(D): Theoretical quantity = 0.0, 95% CI = [-0.0042, 0.0052], Within CI: True
Run 267: N = 500, M = 200, mu_X = -5, sigma_X = 5, mu_epsilon = -5, sigma_1 = 0.5, sigma_2 = 0.4
sigma_sq_1 - E(V): Theoretical quantity = -0.2034, 95% CI = [-0.2099, -0.1946], Within CI: True
sigma_sq_1 - Var(V): Theoretical quantity = 0.0771, 95% CI = [0.074, 0.0797], Within CI: T

sigma_sq_2 - Var(V): Theoretical quantity = 0.0021, 95% CI = [0.002, 0.0021], Within CI: True
E(D): Theoretical quantity = 0.778, 95% CI = [0.7768, 0.7793], Within CI: True
Number of experiments where theoretical quantity lies within bootstrap CI: 
E(V) (sigma_sq_1): 269/280 
Var(V) (sigma_sq_1): 256/280 
E(V) (sigma_sq_2): 268/280 
Var(V) (sigma_sq_2): 243/280 
E(D): 271/280 

Run 281: N = 100, M = 50, mu_X = 0, sigma_X = 0.5, mu_epsilon = 3, sigma_1 = 3, sigma_2 = 1.8
sigma_sq_1 - E(V): Theoretical quantity = 0.0651, 95% CI = [0.066, 0.0698], Within CI: False
sigma_sq_1 - Var(V): Theoretical quantity = 0.005, 95% CI = [0.0047, 0.005], Within CI: True
sigma_sq_2 - E(V): Theoretical quantity = 0.1059, 95% CI = [0.1055, 0.1094], Within CI: True
sigma_sq_2 - Var(V): Theoretical quantity = 0.0049, 95% CI = [0.0046, 0.005], Within CI: True
E(D): Theoretical quantity = 0.0409, 95% CI = [0.0376, 0.0414], Within CI: True
Run 282: N = 100, M = 30, mu_X = 5, sigma_X = 2, mu_epsilon = 0, sigma_1

sigma_sq_1 - Var(V): Theoretical quantity = 0.173, 95% CI = [0.17, 0.1837], Within CI: True
sigma_sq_2 - E(V): Theoretical quantity = 0.9046, 95% CI = [0.8941, 0.9168], Within CI: True
sigma_sq_2 - Var(V): Theoretical quantity = 0.1621, 95% CI = [0.1562, 0.1694], Within CI: True
E(D): Theoretical quantity = 0.1413, 95% CI = [0.1302, 0.1568], Within CI: True
Run 296: N = 5000, M = 2000, mu_X = -1, sigma_X = 1, mu_epsilon = 0, sigma_1 = 0.5, sigma_2 = 0.4
sigma_sq_1 - E(V): Theoretical quantity = -0.1363, 95% CI = [-0.1369, -0.1359], Within CI: True
sigma_sq_1 - Var(V): Theoretical quantity = 0.0003, 95% CI = [0.0003, 0.0004], Within CI: True
sigma_sq_2 - E(V): Theoretical quantity = -0.1034, 95% CI = [-0.104, -0.103], Within CI: True
sigma_sq_2 - Var(V): Theoretical quantity = 0.0003, 95% CI = [0.0003, 0.0003], Within CI: True
E(D): Theoretical quantity = 0.0329, 95% CI = [0.0328, 0.0331], Within CI: True
Run 297: N = 50, M = 20, mu_X = -1, sigma_X = 0.5, mu_epsilon = 5, sigma_1 = 5, si

E(D): Theoretical quantity = 0.4494, 95% CI = [0.4329, 0.4494], Within CI: False
Run 310: N = 500, M = 200, mu_X = -5, sigma_X = 3, mu_epsilon = 1, sigma_1 = 0.2, sigma_2 = 0.04
sigma_sq_1 - E(V): Theoretical quantity = -2.1141, 95% CI = [-2.1173, -2.1085], Within CI: True
sigma_sq_1 - Var(V): Theoretical quantity = 0.0277, 95% CI = [0.0268, 0.0289], Within CI: True
sigma_sq_2 - E(V): Theoretical quantity = -2.1079, 95% CI = [-2.1113, -2.1019], Within CI: True
sigma_sq_2 - Var(V): Theoretical quantity = 0.0276, 95% CI = [0.0266, 0.0288], Within CI: True
E(D): Theoretical quantity = 0.0061, 95% CI = [0.0061, 0.0062], Within CI: True
Run 311: N = 5000, M = 500, mu_X = 5, sigma_X = 0.5, mu_epsilon = -3, sigma_1 = 2, sigma_2 = 0.4
sigma_sq_1 - E(V): Theoretical quantity = 5.2127, 95% CI = [5.2118, 5.213], Within CI: True
sigma_sq_1 - Var(V): Theoretical quantity = 0.0005, 95% CI = [0.0005, 0.0005], Within CI: True
sigma_sq_2 - E(V): Theoretical quantity = 5.6849, 95% CI = [5.6845, 5.6854],

KeyboardInterrupt: 